In [1]:
!pip install sounddevice

In [90]:
import  tensorflow_io as tfio
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sounddevice as sd
import wave
from pydub import AudioSegment

In [ ]:
def record_audio(file_name, duration, sample_rate=16000):
    print(f"Recording for {duration} seconds...")
    
    # Record audio
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
    sd.wait()  # Wait for the recording to complete
    print("Recording complete.")
    
    # Save the recorded audio to a .wav file
    with wave.open(file_name, 'wb') as wf:
        wf.setnchannels(1)  # Mono channel
        wf.setsampwidth(2) 
        wf.setframerate(sample_rate)
        wf.writeframes(audio_data.tobytes())
    
    print(f"Audio saved as {file_name}.")

In [155]:
output_file = "output.wav"
record_duration = 1  # Duration in seconds
record_audio(output_file, record_duration)

Recording for 1 seconds...
Recording complete.
Audio saved as output.wav.


In [156]:
def mono_16k(path):
    file_contents = tf.io.read_file(path)
    wav ,sample_rate = tf.audio.decode_wav(file_contents, desired_channels = 1)
    wav = tf.squeeze(wav, axis = -1)
    sample_rate = tf.cast(sample_rate, dtype= tf.int64)
    #wav = tfio.audio.resample(wav, rate_in = sample_rate, rate_out = 16000)
    return wav

In [157]:

#path='/Users/hotpolarbear/Documents/Programs/Projects/Voice Automation/Voice-Command-Automation-Tensorflow-Arduino/Split-Dataset/test/two/0f250098_nohash_1.wav'
path='output.wav'
wav=mono_16k(path)
tf.shape(wav)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([16000], dtype=int32)>

In [ ]:
def make_spectrogram(wav, sample_rate=16000):
    # Compute STFT
    stft = tf.signal.stft(wav, frame_length=512, frame_step=256, fft_length=512)
    spectrogram = tf.abs(stft)

    # Convert to Mel Spectrogram (128 Mel bins)
    num_mel_bins = 128
    num_spectrogram_bins = tf.shape(spectrogram)[-1]

    # Compute Mel filterbank
    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=num_mel_bins,
        num_spectrogram_bins=num_spectrogram_bins,
        sample_rate=sample_rate,
        lower_edge_hertz=80.0,
        upper_edge_hertz=7600.0
    )

    # Apply Mel filter
    mel_spectrogram = tf.tensordot(spectrogram, tf.cast(mel_filterbank, dtype=tf.float32), axes=(-1, 0))
    mel_spectrogram = tf.expand_dims(mel_spectrogram, axis=-1)  # Shape: (Time, Freq, 1)

    mel_spectrogram = tf.image.resize(mel_spectrogram, (128, 128))

    return mel_spectrogram

In [159]:
spectogram=make_spectrogram(wav)
tf.shape(spectogram)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([128, 128,   1], dtype=int32)>

In [160]:
model=tf.keras.models.load_model('VCA_6M_9336.keras')

In [161]:
config = model.get_config() # Returns pretty much every information about your model
print(config["layers"][0]["config"]["batch_shape"])
print(tf.shape(spectogram))

(None, 128, 128, 1)
tf.Tensor([128 128   1], shape=(3,), dtype=int32)


In [162]:
spectogram = spectogram[tf.newaxis,...]

In [163]:
prediction=model.predict(spectogram)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step


array([[1.4225566e-13, 3.7695944e-10, 1.1320648e-07, 2.2696490e-06,
        9.9999762e-01, 1.8593838e-10, 1.0527939e-13, 1.1638628e-11,
        3.8059753e-15, 2.6923622e-15]], dtype=float32)

In [164]:
o=max(prediction[0])
print(o)
prediction[0]
ind_np = ((np.where(prediction[0] ==o ))[0])
index = (ind_np[0])

0.9999976


In [165]:
labels=['eight', 'five', 'four', 'nine', 'one', 'seven', 'six', 'three','two', 'zero']
print(labels[index])

one


In [170]:
conversion={'zero': 0, 'one': 1, 'two': 2, 'three' : 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}
conversion['nine']

9